# Import Libraries

In [15]:
import sys
import random
import os
import collections
from pathlib import Path
import pickle
import math
#import numpy as np
import sys
import random
import os
import collections
from pathlib import Path
import re
cwd = os.getcwd()

GROUP_FILE_NAME = '2.txt'


# Since we do not take the argument from command line yet
task1_pred_path = cwd + "/task1_predictions"

# Zemberek 

# Docker Installation for Zemberek

brew cask install docker

open --background -a Docker

sudo docker run -d --rm -p 6789:6789 --name zemberek-grpc ryts/zemberek-grpc

In [8]:
%pip install zemberek-grpc

Note: you may need to restart the kernel to use updated packages.


In [9]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys

import grpc

import zemberek_grpc.language_id_pb2 as z_langid
import zemberek_grpc.language_id_pb2_grpc as z_langid_g
import zemberek_grpc.normalization_pb2 as z_normalization
import zemberek_grpc.normalization_pb2_grpc as z_normalization_g
import zemberek_grpc.preprocess_pb2 as z_preprocess
import zemberek_grpc.preprocess_pb2_grpc as z_preprocess_g
import zemberek_grpc.morphology_pb2 as z_morphology
import zemberek_grpc.morphology_pb2_grpc as z_morphology_g

channel = grpc.insecure_channel('localhost:6789')

langid_stub = z_langid_g.LanguageIdServiceStub(channel)
normalization_stub = z_normalization_g.NormalizationServiceStub(channel)
preprocess_stub = z_preprocess_g.PreprocessingServiceStub(channel)
morphology_stub = z_morphology_g.MorphologyServiceStub(channel)

def find_lang_id(i):
    response = langid_stub.Detect(z_langid.LanguageIdRequest(input=i))
    return response.langId

def tokenize(i):
    response = preprocess_stub.Tokenize(z_preprocess.TokenizationRequest(input=i))
    return response.tokens

def normalize(i):
    response = normalization_stub.Normalize(z_normalization.NormalizationRequest(input=i))
    return response

def analyze(i):
    response = morphology_stub.AnalyzeSentence(z_morphology.SentenceAnalysisRequest(input=i))
    return response;

def fix_decode(text):
    """Pass decode."""
    if sys.version_info < (3, 0):
        return text.decode('utf-8')
    else:
        return text

# NLTK Punkt 

**Run the followings from the command line**  
`pip3 install nltk`  
`python3`  
`import nltk`  
`nltk.download()`

Click all packages from the window opened.
Download `punkt`.

In [23]:
import nltk
import pickle
import json
from nltk.tokenize import sent_tokenize

sent_detec = nltk.data.load('tokenizers/punkt/turkish.pickle')

f = open("derlem.txt", encoding="utf-16", errors='ignore')
corpus = f.read()
par_list = corpus.split("\n\n")

par_dict = {}

word_set = set()

for par in par_list:
    start = par.index(" ")
    num = par[:start]
    key = int(num)
    paragraph = par[start+1:]
    sentences = sent_detec.tokenize(par[start+1:])
    lemmas = []
    for sentence in sentences:
        result = analyze(sentence)
        l = {}
        for a in result.results:
            if a.best.pos != 'Punc':
                element = a.best.lemmas[0]
                if a.best.lemmas[0] == 'UNK':
                    element = a.token
                count = l.get(element,0)
                count += 1
                word_set.add(element)
                l[element] = count
        lemmas.append(l)
                
    par_dict[key] = { "paragraph" : paragraph, "sentences" : sentences, "lemmas" : lemmas}

word_dic = {"words" : list(word_set)}
with open('parToSentences.pickle', 'wb') as handle:
    pickle.dump(par_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

    
with open('wordSet.pickle', 'wb') as handle:
    pickle.dump(word_dic, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [32]:
lemmatized_paragraphs = pickle.load(open( cwd + '/parToSentences.pickle', "rb" ))

In [33]:
words = pickle.load(open( cwd + '/wordSet.pickle', "rb" ))

In [34]:
aa = 0
for e in lemmatized_paragraphs:
    print(lemmatized_paragraphs[e])
    aa += 1
    if aa == 5:
        break

{'paragraph': 'Madenler, yaşamımızın birçok alanında çeşitli amaçlarla işleyerek kullandığımız doğal kaynaklardır. İnşaat, otomotiv, beyaz eşya, mutfak eşyaları, elektrikli ve elektronik eşya üretimi gibi birçok alanda madenler kullanılmaktadır.', 'sentences': ['Madenler, yaşamımızın birçok alanında çeşitli amaçlarla işleyerek kullandığımız doğal kaynaklardır.', 'İnşaat, otomotiv, beyaz eşya, mutfak eşyaları, elektrikli ve elektronik eşya üretimi gibi birçok alanda madenler kullanılmaktadır.'], 'lemmas': [{'maden': 1, 'yaşam': 1, 'birçok': 1, 'alan': 1, 'çeşit': 1, 'amaç': 1, 'işle': 1, 'kullan': 1, 'doğal': 1, 'kaynakla': 1}, {'inşaat': 1, 'otomotiv': 1, 'beyaz': 1, 'eşya': 3, 'mutfak': 1, 'elektrik': 1, 've': 1, 'elektronik': 1, 'üretim': 1, 'gibi': 1, 'birçok': 1, 'alan': 1, 'maden': 1, 'kullan': 1}]}
{'paragraph': 'Türkiye, maden çeşidi bakımından zengin bir ülkedir. Bu maden yataklarının önemli bir kısmı işlenmektedir. Bir maden yatağı işletmeye açılırken o madenin toplam miktarı 

In [22]:
for k in lemmatized_paragraphs:
    
    print([x.keys() for x in lemmatized_paragraphs[k]["lemmas"]])

[dict_keys(['maden', 'yaşam', 'birçok', 'alan', 'çeşit', 'amaç', 'işle', 'kullan', 'doğal', 'kaynakla']), dict_keys(['inşaat', 'otomotiv', 'beyaz', 'eşya', 'mutfak', 'elektrik', 've', 'elektronik', 'üretim', 'gibi', 'birçok', 'alan', 'maden', 'kullan'])]
[dict_keys(['türkiye', 'maden', 'çeşit', 'bakım', 'zengin', 'bir', 'ülke']), dict_keys(['bu', 'maden', 'yatak', 'önem', 'bir', 'kısım', 'işle']), dict_keys(['bir', 'maden', 'yatak', 'işle', 'aç', 'o', 'toplam', 'miktar', 'yani', 'rezerv', 'belirle']), dict_keys(['maden', 'iç', 'saf', 'metal', 'oran', 'ol', 'tenör', 'de', 'bir', 'yatak', 'işle', 'aç', 'önem', 'taşı', 'özellik']), dict_keys(['çünkü', 'miktar', 'fazla', 'ol', 'da', 'tenörü', 'çok', 'düşük', 'yatak', 'işle', 'aç']), dict_keys(['bu', 'yan', 'sıra', 'işle', 'maliyet', 'hesapla', 've', 'kar', 'ol', 'yatak'])]
[dict_keys(['ülke', 'çıkar', 'maden', 'bazı', 'metalik', 'demir', 'bakır', 'krom', 'kurşun', 've', 'çinko', 'gibi']), dict_keys(['bor', 'kükürt', 'asbest', 'fosfat', 'gi

[dict_keys(['iklim', 'hayvan', 'göç', 'hareket', 'de', 'etkile']), dict_keys(['örnek', 'yağ', 'yaz', 'mevsim', 'yo\xadğunlaştığı', 'savan', 'iklim', 'bölge', 'yaşa', 'hayvan', 'kurak', 'geçen', 'kış', 'göç', 'etmekte\xaddir']), dict_keys(['benzer', 'şekil', 'orta', 'kuşak', 'birçok', 'kuş', 'tür', 'de', 'soğuk', 'dönem', 'ekvator', 'sıcak', 'dö\xadnemlerde', 'i', 'kutup', 'doğru', 'göç', 'et', 'gör', 'mümkün'])]
[dict_keys(['bitki', 'sıcaklık', 'azal', 'bağ', 'ol', 'ekvator', 'kutup', 'doğru', 'geniş', 'yapraklı', 'karışık', 've', 'iğne', 'ağaç', 'şekil', 'kuşak', 'oluş']), dict_keys(['orta', 'kuşak', 'bitki', 'kış', 'yaprak', 'dök']), dict_keys(['kurak', 'iklim', 'bölge', 'bazı', 'bitki', 'su', 'kayıp', 'yaşa', 'ad', 'sert', 'kalın', 'veya', 'tüy', 'yaprak', 'sistem', 'sahip', 'bir', 'şekil', 'dizayn', 'et'])]
[dict_keys(['ışık', 'miktar', 've', 'süre', 'canlı', 'yaşam', 'etkile', 'diğer', 'bir', 'faktör']), dict_keys(['bitki', 'çimle', 'çiçek', 'aç', 've', 'geliş\xadme', 'süreç', 'ış

[dict_keys(['30°', 'kuzey', 'enlem', 'Türkiye', "'", 'nin', 'batı', 'oluşan', 'azor', 'dinamik', 'yüksek', 'basınç', 'merkez', 'yaz', 'ay', 'büyük', 'bir', 'kısım', 'kuraklığının', 'yaşanmasına', 'neden', 'ol'])]
[dict_keys(['basra', 'termik', 'Alçak', 'Basınç', 'merkez'])]
[dict_keys(['muson', 'sistem', 'bir', 'parçası', 'ol', 'basra', 'Körfezi', 'çevresinde', 'oluşan', 'termik', 'alçak', 'basınç', 'merkez', 'Türkiye', "'", 'nin', 'güneydoğu', 'kesim', 'yaz', 'ay', 'sıcak', 've', 'kuru', 'hava', 'koşullarının', 'görülmesine', 'neden'])]
[dict_keys(['Türkiye', "'", 'de', 'etki', 'ol', 'yerel', 'Rüzgârlar'])]
[dict_keys(['Türkiye', 'çeşitli', 'yerel', 'rüzgârların', 'etki', 'alt', 'kal', 'bir', 'ülkedir']), dict_keys(['mutlak', 'konum', 'neden', 'kuzey', 'yönünden', 'es', 'rüzgârlar', 'sıcaklığı', 'azal', 'güneyden', 'esenle', 'yükseltir']), dict_keys(['Türkiye', "'", 'de', 'etki', 'ol', 'rüzgârların', 'bazı', 'yerel', 'topoğrafik', 'özelliklere', 'bağlı', 'oluşurken', 'i', 'nin', 'gene

# Helper Functions

In [50]:
def containArr(arr1, arr2):
    """returns true if at least one element from arr1 is contained by arr2"""
    for element in arr1:
        if element in arr2:
            return True
    return False

def paragraph_lemma_dic(par_id):
    key = par_id
    paragraph = lemmatized_paragraphs[par_id]["paragraph"]
    sentences = sent_detec.tokenize(paragraph)
    dic = {}
    for sentence in sentences:
        result = analyze(sentence)
        kk = set()
        for a in result.results:
            if a.best.pos != 'Punc':
                element = a.best.lemmas[0]
                if a.best.lemmas[0] == 'UNK':
                    element = a.token
                kk.add(element)
        dic[sentence] = list(kk)
                
    return dic



In [56]:
paragraph_lemma_dic(226)

{'Madenler, yaşamımızın birçok alanında çeşitli amaçlarla işleyerek kullandığımız doğal kaynaklardır.': ['işle',
  'kullan',
  'birçok',
  'çeşit',
  'maden',
  'alan',
  'yaşam',
  'amaç',
  'kaynakla',
  'doğal'],
 'İnşaat, otomotiv, beyaz eşya, mutfak eşyaları, elektrikli ve elektronik eşya üretimi gibi birçok alanda madenler kullanılmaktadır.': ['kullan',
  'beyaz',
  'birçok',
  'eşya',
  'inşaat',
  'mutfak',
  'maden',
  'üretim',
  'gibi',
  'alan',
  'elektronik',
  've',
  'otomotiv',
  'elektrik']}

## Read soru_gruplari.txt

In [16]:
# Change the encoding of the input file to latin-1
file_content = ""
with open("soru_gruplari.txt", encoding="utf-16") as f:
    file_content = f.read()
# Read the file and append all of the lines
question_groups = file_content.split("\n\n")

In [17]:
question_groups_dict = {}
for group in question_groups:
    lines = group.split("\n")
    answer_id = ""
    group_dict = collections.defaultdict(dict)
    for line in lines:
        ID = re.findall("(.*):", line)[0]
        sentence = re.findall(": (.*)", line)[0]
        if ID[0] == 'S':  # if the sentence is a question
            group_dict["questions"][ID] = sentence
        elif ID[0] == 'C':  # if the sentence is the answer
            group_dict["answer"] = sentence
            answer_id = ID
        else:  # if the sentence is the id of related paragraph
            group_dict["paragraph"] = sentence
    question_groups_dict[answer_id] = group_dict

In [18]:
paragraphs_and_terms = {}
all_terms = set()
for paragraph in lemmatized_paragraphs:
    terms = [x for x in lemmatized_paragraphs[paragraph][lemmas]]
    all_terms.update(terms)
    paragraphs_and_terms[paragraph_id] = terms
all_terms = list(all_terms)

In [31]:
for el in lemmatized_paragraphs[226]["lemmas"]:
    for a in el:
        print(a)


maden
yaşam
birçok
alan
çeşit
amaç
işle
kullan
doğal
kaynakla
inşaat
otomotiv
beyaz
eşya
mutfak
elektrik
ve
elektronik
üretim
gibi
birçok
alan
maden
kullan


In [36]:
# Compute term frequency for each term and paragraph
tf = {}
for paragraph_id in lemmatized_paragraphs:
    tf_par = {}
    for el in lemmatized_paragraphs[paragraph_id]["lemmas"]:
        for val in el:
            tf_val = tf_par.get(val, 0)
            tf_val += 1
            tf_par[val] = tf_val
    tf[paragraph_id] = tf_par
    
with open('tf.pickle', 'wb') as handle:
    pickle.dump(tf, handle, protocol=pickle.HIGHEST_PROTOCOL)


{226: {'maden': 2, 'yaşam': 1, 'birçok': 2, 'alan': 2, 'çeşit': 1, 'amaç': 1, 'işle': 1, 'kullan': 2, 'doğal': 1, 'kaynakla': 1, 'inşaat': 1, 'otomotiv': 1, 'beyaz': 1, 'eşya': 1, 'mutfak': 1, 'elektrik': 1, 've': 1, 'elektronik': 1, 'üretim': 1, 'gibi': 1}, 227: {'türkiye': 1, 'maden': 4, 'çeşit': 1, 'bakım': 1, 'zengin': 1, 'bir': 4, 'ülke': 1, 'bu': 2, 'yatak': 5, 'önem': 2, 'kısım': 1, 'işle': 5, 'aç': 3, 'o': 1, 'toplam': 1, 'miktar': 2, 'yani': 1, 'rezerv': 1, 'belirle': 1, 'iç': 1, 'saf': 1, 'metal': 1, 'oran': 1, 'ol': 3, 'tenör': 1, 'de': 1, 'taşı': 1, 'özellik': 1, 'çünkü': 1, 'fazla': 1, 'da': 1, 'tenörü': 1, 'çok': 1, 'düşük': 1, 'yan': 1, 'sıra': 1, 'maliyet': 1, 'hesapla': 1, 've': 1, 'kar': 1}, 228: {'ülke': 2, 'çıkar': 1, 'maden': 3, 'bazı': 2, 'metalik': 2, 'demir': 1, 'bakır': 1, 'krom': 1, 'kurşun': 1, 've': 1, 'çinko': 1, 'gibi': 2, 'bor': 1, 'kükürt': 1, 'asbest': 1, 'fosfat': 1, 'i': 1, 'değil': 1, 'şimdi': 1, 'başlıca': 1, 'incele': 1}, 229: {'demir': 4, 'çelik':

In [40]:
# Compute document frequency for each term
df = {}
for term in word_dic["words"]:
    df[term] = sum([1 for paragraph_id in tf if term in tf[paragraph_id]])

Tf.idf score for a given query (q) and the set of paragraphs (documents - d) is $\sum\limits_{t\in q\cap d} tf.idf_{t,d}$ where

tf score = $\begin{cases}
  1 + log_{10}tf_{t, d}, & \text{if $tf_{t, d} > 0$}\\
  0, & \text{otherwise}
\end{cases}  $

and

idf score = $log_{10}(N / df_{t})$

In [43]:
import math
import collections

# Compute tf-idf scores for each paragraph
weights = collections.defaultdict(list)
for paragraph_id in lemmatized_paragraphs:
    for term in word_dic["words"]:
        if term in tf[paragraph_id]:
            weights[paragraph_id].append((1 + math.log10(tf[paragraph_id][term]) * math.log10(len(lemmatized_paragraphs)/df[term])))
        else:
            weights[paragraph_id].append(0)

## Predict paragraph

In [46]:
def predict_related_paragraph(question):

    res = analyze(question)
    query_terms = []
    for r in res.results:
        query_terms.append(r.best.lemmas[0])
        
    query_term_indices = [word_dic["words"].index(term) for term in query_terms if term in word_dic["words"]]

    tf_idf = {}
    for paragraph_id in weights:
        tf_idf[paragraph_id] = sum([weight for index, weight in enumerate(weights[paragraph_id]) if index in query_term_indices])

    related_paragraphs = sorted(tf_idf.items(), key=lambda x: x[1], reverse=True)
    
    return related_paragraphs[0][0]

## Predict answer

In [ ]:
def find_answer(question, answerSentence):
    analysis_result = analyze(answerSentence)
    answer_words = []
    answer_types = []
    a_dic = {}
    index = 0
    for a in analysis_result.results:
        best = a.best
        answer_words.append(best.lemmas[0])
        answer_types.append(best.pos)
        b = []
        b.append(index)
        b.append(a.token)
        b.append(best.lemmas[0])
        b.append(best.pos)
        index += 1
        a_dic[best.lemmas[0]] = b
        lemmas = ""
        for l in best.lemmas:
            lemmas = lemmas + " " + l
        print("Word = " + a.token + ", Lemmas = " + lemmas + ", POS = [" + best.pos + "], Full Analysis = {" + best.analysis + "}")
        
    question_words = []
    question_types = {}
    q = analyze(question)
    #print(a_dic)
    for a in q.results:
        best = a.best
        question_words.append(best.lemmas[0])
        question_types[best.lemmas[0]]=(best.pos)
        lemmas = ""
        for l in best.lemmas:
            lemmas = lemmas + " " + l
        print("Word = " + a.token + ", Lemmas = " + lemmas + ", POS = [" + best.pos + "], Full Analysis = {" + best.analysis + "}")
        
    l1 = (list(set(question_words)-set(answer_words)))    
    l2 = (list(set(answer_words)-set(question_words)))
    
    l2.sort(key=lambda x: a_dic[x][0], reverse=False)
    
    res = []
    if containArr(["kaç","yıl","kadar","sene"], question_words):
        res = ([a_dic[x][1] for x in l2 if a_dic[x][3] == 'Num'])
    elif containArr(["bölge","ülke","kıta","il", "ilçe", "köy", "eyalet"], question_words):
        res = ([a_dic[x][1]  for x in l2 if a_dic[x][3] == 'Noun'])
    elif "nasıl" in question_words:

        res = ([a_dic[x][1]  for x in l2 if a_dic[x][3] == 'Adj' or a_dic[x][3] == 'Adv' or a_dic[x][3] == 'Verb' ])
    elif containArr(["ad","isim"], question_words): #bakılacak
        res = ([a_dic[x][1]  for x in l2 if a_dic[x][3] == 'Noun'])
    elif "kim" in question_words:
        res = ([a_dic[x][1]  for x in l2 if a_dic[x][3] == 'Noun'])
    elif containArr(["mı","mi","mu","mü"],question_words):
        if len([a_dic[x][1] for x in l2 if a_dic[x][3] == 'Verb']) != 0:
            res = ([a_dic[x][1] for x in answer_words if a_dic[x][3] == 'Verb'])
        else:
            res = ([a_dic[x][1]  for x in l2 if a_dic[x][3] =='Adv'])
    else:
        res = [a_dic[x][1] for x in l2 if (a_dic[x][3] != "Postp" and a_dic[x][3] != "Conj" and a_dic[x][3] != "Punc" and a_dic[x][3] != "Det")]
    print("cevap ", res)
    return res

# Predict sentence

In [78]:
def predict_sentence(paragraph_id, question):
    """
    Predict which sentence in a paragraph is related to a question
    """
    sentences_and_terms = paragraph_lemma_dic(paragraph_id)
    
    kk = set()
    for s in list(sentences_and_terms.values()):
        kk.update(s)
    kk = list(kk)
    # Compute document frequency for each term
    df = {}
    for term in kk:
        df[term] = sum([1 for sentence in sentences_and_terms if term in sentences_and_terms[sentence]])

    # Compute tf-idf scores for each paragraph
    weights = collections.defaultdict(list)
    for i, sentence in enumerate(lemmatized_paragraphs[paragraph_id]['sentences']):
        for term in kk:
            if term in lemmatized_paragraphs[paragraph_id]['lemmas'][i]:
                weights[sentence].append((1 + lemmatized_paragraphs[paragraph_id]['lemmas'][i][term]) * len(sentences_and_terms)/df[term])
            else:
                weights[sentence].append(0)
    
    res = analyze(question)
    query_terms = []
    for r in res.results:
        query_terms.append(r.best.lemmas[0])
        
    query_term_indices = [kk.index(term) for term in query_terms if term in kk]

    tf_idf = {}
    for sentence in weights:
        tf_idf[sentence] = sum([weight for index, weight in enumerate(weights[sentence]) if index in query_term_indices])

    related_sentences = sorted(tf_idf.items(), key=lambda x: x[1], reverse=True)
    return related_sentences[0][0]

# Write predicted results to files

In [ ]:
related_paragraphs = []
related_sentences = []
answers = []
with open(cwd + '/test_questions.txt', 'r', encoding="utf-16") as f:
    questions = f.readlines()
    for question in questions:
        paragraph_id = predict_related_paragraph(question)
        sentence = predict_sentence(paragraph_id, question)
        print("q ",question)
        answer = find_answer(question, sentence)
        print("s ", sentence)
        print("a ",answer)
        related_paragraphs.append(str(paragraph_id))
        related_sentences.append(sentence)
        answers.append(answer)

q  ABD bağımsızlığını hangi yıl kazanmıştır?

Word = ABD, Lemmas =  abd, POS = [Noun], Full Analysis = {[ABD:Noun,Abbrv] abd:Noun+A3sg}
Word = bağımsızlığını, Lemmas =  bağımsız bağımsızlık, POS = [Noun], Full Analysis = {[bağımsız:Adj] bağımsız:Adj|lığ:Ness→Noun+A3sg+ın:P2sg+ı:Acc}
Word = hangi, Lemmas =  hangi, POS = [Adj], Full Analysis = {[hangi:Adj] hangi:Adj}
Word = yıl, Lemmas =  yıl, POS = [Noun], Full Analysis = {[yıl:Noun,Time] yıl:Noun+A3sg}
Word = kazanmıştır, Lemmas =  kazan, POS = [Verb], Full Analysis = {[kazanmak:Verb] kazan:Verb+mış:Narr+A3sg+tır:Cop}
Word = ?, Lemmas =  ?, POS = [Punc], Full Analysis = {[?:Punc] ?:Punc}
cevap  []
s  1777’de bağımsızlığını kazanan ABD’nin sürekli göç alması nüfusunu artırmış ve doğal kaynakların daha kolay işlenmesini sağlamıştır.
a  None
q  Almanya'da hangi iklim şartları hakimdir?

Word = Almanya'da, Lemmas =  almanya, POS = [Noun], Full Analysis = {[Almanya:Noun,Prop] almanya:Noun+A3sg+da:Loc}
Word = hangi, Lemmas =  hangi, POS = [A

In [ ]:
# Uncomment the following line if you are running this for the first time
# os.mkdir(task1_pred_path)

with open(Path(task1_pred_path) / GROUP_FILE_NAME, 'w+', encoding='utf16') as f:
    f.write('\n'.join(related_paragraphs))

# TODO: complete task 2    

with open(Path(task2_pred_path) / GROUP_FILE_NAME, 'w', encoding='utf16') as f:
    f.write('\n'.join(answers))


# Extracting the Question from The Paragraph
Assuming that the right sentence from the right paragraph is given for the question.

In [193]:
find_answer("istanbulun başkentini yazınız?", "istanbulun başkenti ankarada yer alır.")

{'istanbul': [0, 'istanbulun', 'istanbul', 'Noun'], 'başkent': [1, 'başkenti', 'başkent', 'Noun'], 'ankara': [2, 'ankarada', 'ankara', 'Noun'], 'yer': [3, 'yer', 'yer', 'Noun'], 'al': [4, 'alır', 'al', 'Verb'], '.': [5, '.', '.', 'Punc']}
Word = istanbulun, Lemmas =  istanbul, POS = [Noun], Full Analysis = {[İstanbul:Noun,Prop] istanbul:Noun+A3sg+un:Gen}
Word = başkentini, Lemmas =  başkent, POS = [Noun], Full Analysis = {[başkent:Noun] başkent:Noun+A3sg+i:P3sg+ni:Acc}
Word = yazınız, Lemmas =  yazı, POS = [Noun], Full Analysis = {[yazı:Noun] yazı:Noun+A3sg+nız:P2pl}
Word = ?, Lemmas =  ?, POS = [Punc], Full Analysis = {[?:Punc] ?:Punc}
['ankara', 'yer', 'al', '.']

--------------

['ankarada', 'yer', 'alır']
